In [ ]:
from floodlight.io import dfl
from floodlight.io import opta
import numpy as np
import pandas as pd
import fileinput
import os
import csv
import glob
from lxml import etree
import pickle

In [ ]:
# f24-22-2016-861416
# needed to check if opta reports an actual red card (or includes a qualifier signaling a correction of the decision for example)

Events = opta.read_event_data_xml('EventData/f24-22-2016-861416-eventdetails.xml')[0]      # read the events

data = Events['HT2']['Home'].events
cards = data[data.eID == 17]
cards

## Overview of games in event data files

In [ ]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
opta_mns = []    # opta name of the match
home_teams =  []    # home teams
away_teams =  []    # away teams
dates = []           # date of the match
 
halfs = ['HT1', 'HT2']     # halfs for loop
teams = ['Home', 'Away']   # teams for loop
bools_c = []                # store bools of cards on match level
bools_r = []                # store bools of retired on match level

retired = []             # store bools of retired one for each match
red_card = []            # store bools of red_cards one for each match

dirHome1 = []
dirHome2 = []
dirAway1 = []
dirAway2 = []



ROOT = os.getcwd() # returns current working directory of a process.

ev_data = (r"Eventdata") # creates a raw string which treats backslash (\) as a literal character. 

for dir in os.listdir(ev_data):
    if dir.startswith('Spieltag'):
        md = dir
        ST = md.replace('Spieltag ', '')# get the numerical matchday
        loc = os.path.join(ev_data, dir) 
        # print(loc)
        for eve_file in os.listdir(loc):
            # print(eve_file)
            if eve_file.startswith('f'):
                opta_mn = eve_file.replace('-eventdetails.xml', '')
                tree = etree.parse(str('/'.join([loc, eve_file])))
                root = tree.getroot()
                matchinfo = root.xpath("Game")[0].attrib
                matchdays_numeric.append(ST)
                opta_mns.append(opta_mn)
                home_teams.append(matchinfo['home_team_name'])
                away_teams.append(matchinfo['away_team_name'])
                dates.append(matchinfo['game_date'])
                
                Events = opta.read_event_data_xml(str('/'.join([loc, eve_file])))[0]      # read the events
                halfs = ['HT1', 'HT2']
                teams = ['Home', 'Away']
                bools_c = []
                bools_r = []
                
                
                for h in halfs:
                    for t in teams:
                        e_data = Events[h][t].events

                        # any one being subbed off due to injury when no subs are left in this match?
                        ret = e_data[e_data.eID == 20]
                        if len(ret) > 0:
                            bools_r.append(True)
                        else:
                            bools_r.append(False)

                        # any red cards or second yellows in this match?
                        cards = e_data[e_data.eID == 17]
                        bools_c.append(any(cards.qualifier.str.contains('32:')) or any(cards.qualifier.str.contains('33:')))
                        
                        # match direction for that team
                        di = e_data[e_data.eID == 32]
                        if len(di) > 1:
                            print('More than one start for that half')
                        else: 
                            if any(di.qualifier.str.contains("'Right to Left'")):
                                direction = 'Right to Left'
                            elif any(di.qualifier.str.contains("'Left to Right'")):
                                direction = 'Left to Right'
                            else:
                                direction = 'No direction found'
                            if h == 'HT1':
                                if t == 'Home':
                                    dirHome1.append(direction)
                                elif t == 'Away':
                                    dirAway1.append(direction)
                            elif h == 'HT2':
                                if t == 'Home':
                                    dirHome2.append(direction)
                                elif t == 'Away':
                                    dirAway2.append(direction)
                                    

                red_card.append(any(bools_c))    # in any of the 2 teams and two halves a red card
                retired.append(any(bools_r))       # in any of the 2 teams and two halves a retirement
 

                
                
                
opta_game_overview = pd.DataFrame(list(zip(dates, home_teams, away_teams, matchdays_numeric, opta_mns, red_card, retired, dirHome1, dirHome2, dirAway1, dirAway2)),
                             columns=['Opta_Date', 'Opta_Home', 'Opta_Away', 'Opta_Matchday', 'Opta_Name', 'Red_card', 'Retired', 'Direction_Home_HT1', 'Direction_Home_HT2', 'Direction_Away_HT1', 'Direction_Away_HT2'])
opta_game_overview

# events missing for one game on matchday 2

In [ ]:
opta_game_overview.Opta_Matchday.value_counts()

## Overview of games in match information files

In [ ]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
DFL_mns = []    # dfl name of the match
home_teams =  []    # home teams
away_teams =  []    # away teams
dates = []           # date of the match
name_match = []      # does the match name in the file name and the match information match?!
end_scores = []       # end score of the match 




ROOT = os.getcwd() # returns current working directory of a process.

mi_data = (r"Matchinfo") # creates a raw string which treats backslash (\) as a literal character. 


for mi_file in os.listdir(mi_data):
    #print(mi_file)
    if mi_file.startswith('DFL_'):
        DFL_nm = mi_file[-18:-4]
        #print(DFL_nm)
        tree = etree.parse(str('/'.join([mi_data, mi_file])))
        root = tree.getroot()
        team_informations = root.find("MatchInformation").find("Teams")
        mi_home = root.find("MatchInformation").find("General").get("HomeTeamName")
        home_teams.append(mi_home)
        mi_away = root.find("MatchInformation").find("General").get("GuestTeamName")
        away_teams.append(mi_away)
        mi_mn = root.find("MatchInformation").find("General").get("MatchId")
        DFL_mns.append(mi_mn)
        mi_date = root.find("MatchInformation").find("General").get("PlannedKickoffTime")
        dates.append(mi_date)
        score = root.find("MatchInformation").find("General").get("Result")
        end_scores.append(score)
        mi_Matchday = root.find("MatchInformation").find("General").get("MatchDay")
        matchdays_numeric.append(mi_Matchday)
        boolean = DFL_nm == mi_mn
        name_match.append(boolean)
        
mi_game_overview = pd.DataFrame(list(zip(dates, home_teams, away_teams, end_scores, matchdays_numeric, DFL_mns, name_match)),
                             columns=['MI_Date', 'MI_Home', 'MI_Away', 'Final_Score', 'MI_Matchday', 'MI_Name', 'NameCorrect?'])

# mi_game_overview
mi_game_overview = mi_game_overview[:306] # remove relegation matches!
mi_game_overview

In [ ]:
mi_game_overview['MI_Matchday'].value_counts()
# top!

## Overview of games in position data files

In [ ]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
DFL_mns = []        # dfl name of the match




ROOT = os.getcwd() # returns current working directory of a process.

pos_data = (r"PositionData") # creates a raw string which treats backslash (\) as a literal character. 

for pos_file in os.listdir(pos_data):
    if pos_file.startswith('DFL_'):
        DFL_nm = pos_file[-18:-4]
        DFL_mns.append(DFL_nm)

                
pos_game_overview = pd.DataFrame(list(zip(DFL_mns)),
                             columns=['Pos_Name'])

pos_game_overview

# at least one missing
# 3 missing because 2 of these are relegation matches

In [ ]:
dfl_overview = mi_game_overview
dfl_overview['Pos_Name'] = np.nan

for row in range(len(pos_game_overview)):
    name = pos_game_overview.Pos_Name[row]
    # print(name)
    if (mi_game_overview['MI_Name'].eq(name)).any():
        index = np.where(mi_game_overview.MI_Name == name)[0][0]
        # print(index)
        dfl_overview.loc[index, 'Pos_Name'] = name
    else:
        print(name)
    
dfl_overview

# DFL-MAT-0027W0 in pos but not mi = relegation
# DFL-MAT-0027W1 in pos but not mi = relegation

In [ ]:
dfl_overview[80:90]
# 85 - 87
# DFL-MAT-0027AA HSV - BVB 10
# DFL-MAT-0027AB Ingolstadt - FCA 10
# DFL-MAT-0027AC Frankfurt - 1.FC


In [ ]:
dfl_overview.rename(columns={"MI_Home": "Home", "MI_Away": "Away", 'MI_Matchday': 'Matchday'}, inplace=True)
opta_game_overview.rename(columns={"Opta_Home": "Home", "Opta_Away": "Away", 'Opta_Matchday': 'Matchday'}, inplace=True)

game_overview = pd.merge(dfl_overview, opta_game_overview, on=["Home", "Away", "Matchday"], how='outer')
game_overview.to_csv('Output/GameOverview.csv')
game_overview

In [ ]:
game_overview[250:306]
# SV Werder Bremen	FC Augsburg	2	DFL-MAT-00278B	 missing event data?!

# Missing Data!

### EventData
Matchday 2 - SV Werder Bremen vs FC Augsburg - DFL_Name = DFL-MAT-00278B

### PositionData
Matchday 10 - Hamburger SV vs. Borussia Dortmund - DFL_Name = DFL-MAT-0027AA <br>
Matchday 10 - FC Ingolstadt 04 vs. FC Augsburg - DFL_Name = DFL-MAT-0027AB <br>
Matchday 10 - Eintracht Frankfurt vs. 1. FC Köln - DFL_Name = DFL-MAT-0027AC <br>

In [ ]:
# exclude missing data

game_overview[game_overview.Pos_Name.isna()]
game_overview[game_overview.Opta_Name.isna()]

game_overview2 = game_overview[game_overview.Pos_Name.notna() & game_overview.Opta_Name.notna()].reset_index()
game_overview2.to_csv('Output/GameOverview2.csv')
game_overview2

In [ ]:
game_names_Opta = {game_overview2.Opta_Name[i]: game_overview2.MI_Name[i] for i in range(len(game_overview2.Opta_Name))}

with open("Output/game_names_Opta.pkl", "wb") as cdict:
    pickle.dump(game_names_Opta, cdict)
print(game_names_Opta)

game_MD_Opta = {game_overview2.Opta_Name[i]: game_overview2.Matchday[i] for i in range(len(game_overview2.Opta_Name))}
print(game_MD_Opta)

with open("Output/game_MD_Opta.pkl", "wb") as cdict:
    pickle.dump(game_MD_Opta, cdict)



In [ ]:
game_names_DFL = {game_overview2.MI_Name[i]: game_overview2.Opta_Name[i] for i in range(len(game_overview2.MI_Name))}

with open("Output/game_names_DFL.pkl", "wb") as cdict:
    pickle.dump(game_names_DFL, cdict)
game_names_DFL